In [1]:
import numpy as np
import matplotlib.pyplot as plt
import visualization
import os
from gta_math import points_to_homo, ndc_to_view, construct_proj_matrix, view_to_world, construct_view_matrix
from visualization import load_depth, load_stencil, save_pointcloud_csv, bbox_from_string
import progressbar
from joblib import Parallel, delayed
from configparser import ConfigParser
from PIL import Image
import pickle
import json
import time
from shutil import copyfile

In [44]:
ini_file = "gta-postprocessing.ini"
visualization.multi_page = False
visualization.ini_file = ini_file
visualization.use_cache = False

conn = visualization.get_connection()
cur = conn.cursor()

CONFIG = ConfigParser()
CONFIG.read(ini_file)
in_directory = CONFIG["Images"]["Tiff"]
out_directory = r'D:\output-datasets\offroad-4'

# for some datasets we don't need entities, which produces lot smaller json files
include_entities = False

In [45]:
#run_id = 19
#run_id = 3677
run_id = 4006

cur.execute("""SELECT snapshot_id, imagepath, cam_near_clip, camera_fov, width, height, timestamp, timeofday, width, height, \
      ARRAY[st_x(camera_relative_rotation), st_y(camera_relative_rotation), st_z(camera_relative_rotation)] as camera_relative_rotation, \
      ARRAY[st_x(camera_relative_position), st_y(camera_relative_position), st_z(camera_relative_position)] as camera_relative_position, \
      ARRAY[st_x(camera_pos), st_y(camera_pos), st_z(camera_pos)] as camera_pos, \
      ARRAY[st_x(camera_rot), st_y(camera_rot), st_z(camera_rot)] as camera_rot, \
      currentweather \
      FROM snapshots \
      WHERE run_id = {}
      ORDER BY timestamp ASC \
    """.format(run_id))

results = []
for row in cur:
    res = dict(row)
    if res['camera_fov'] == 0 and res['cam_near_clip'] == 0:
        continue  # somehow malformed data, skipping them
    res['camera_rot'] = np.array(res['camera_rot'])
    res['camera_pos'] = np.array(res['camera_pos'])
    res['camera_relative_rotation'] = np.array(res['camera_relative_rotation'])
    res['camera_relative_position'] = np.array(res['camera_relative_position'])
    res['view_matrix'] = construct_view_matrix(res['camera_pos'], res['camera_rot'])
    res['proj_matrix'] = construct_proj_matrix(res['height'], res['width'], res['camera_fov'], res['cam_near_clip'])
    results.append(res)

print('There are {} snapshots'.format(len(results)))

There are 62892 snapshots


In [46]:
def get_base_name(name):
    return os.path.basename(os.path.splitext(name)[0])


def get_main_image_name(cameras):
    for cam in cameras:
        # this is the main camera
        if np.array_equal(cam['camera_relative_rotation'], [0, 0, 0]):
            return cam['imagepath']
    raise Exception('no main image')


def load_entities_data(snapshot_id):
    conn = visualization.get_connection_pooled()
    cur = conn.cursor()
    
    # start = time.time()

    cur.execute("""SELECT bbox, \
        ARRAY[st_x(pos), st_y(pos), st_z(pos)] as pos, \
        ARRAY[st_x(rot), st_y(rot), st_z(rot)] as rot, \
        ARRAY[st_xmin(bbox3d), st_xmax(bbox3d), st_ymin(bbox3d), st_ymax(bbox3d), st_zmin(bbox3d), st_zmax(bbox3d)] as bbox3d, \
         type, class, handle, snapshot_id \
        FROM detections \
        WHERE snapshot_id = '{}' \
        """.format(snapshot_id))

    # end = time.time()
    # print('time to load from db', end - start)
    # start = time.time()

    # print(size)
    results = []
    for row in cur:
        res = dict(row)
        res['model_sizes'] = np.array(res['bbox3d'])
        res['bbox'] = bbox_from_string(res['bbox'])
        res['pos'] = np.array(res['pos'])
        res['rot'] = np.array(res['rot'])
        results.append(res)

    # end = time.time()
    # print('time to convert arrays to numpy', end - start)
    # start = time.time()

    return results


def convert_rgb(in_directory, out_directory, out_name, name, out_format):
    outfile = os.path.join(out_directory, "{}.{}".format(out_name, out_format))
    # print(outfile)
    if os.path.exists(outfile):
        return

    try:
        infile = os.path.join(in_directory, name)
        im = Image.open(infile)
        im = im.convert(mode="RGB")
        im.save(outfile)
    except OSError:
        # print("Skipping invalid file {}".format(name))
        return


def convert_depth(in_directory, out_directory, out_name, name, out_format):
    outfile = os.path.join(out_directory, "{}.{}".format(out_name, out_format))
    # print(outfile)
    if os.path.exists(outfile):
        return

    try:
        infile = os.path.join(in_directory, name)
        depth = load_depth(name)
        # print('depth min before: ', np.min(depth))
        # print('depth max before: ', np.max(depth))
        depth = depth * np.iinfo(np.uint16).max
        # print('depth min after: ', np.min(depth))
        # print('depth max after: ', np.max(depth))
        im = Image.fromarray(depth.astype(np.int32), mode="I")
        im.save(outfile)
    except OSError:
        # print("Skipping invalid file {}".format(name))
        return


def convert_stencil(in_directory, out_directory, out_name, name, out_format):
    outfile = os.path.join(out_directory, "{}.{}".format(out_name, out_format))
    # print(outfile)
    if os.path.exists(outfile):
        return

    try:
        infile = os.path.join(in_directory, name)
        stencil = load_stencil(name)
        im = Image.fromarray(stencil.astype(np.uint8), mode="L")
        im.save(outfile)
    except OSError:
        # print("Skipping invalid file {}".format(name))
        return


def dump_snapshot_to_dataset(in_directory, out_directory, res):
    if 'pbar' in globals() and 'counter' in globals():
        global counter
        counter += 1
        pbar.update(counter)

    start = time.time()
    name = res['imagepath']
#    convert_rgb(in_directory, out_directory, get_base_name(name), get_base_name(name)+'.tiff', 'png')
    convert_rgb(in_directory, out_directory, get_base_name(name), get_base_name(name)+'.tiff', 'jpg')
    convert_depth(in_directory, out_directory, get_base_name(name)+'-depth', get_base_name(name), 'png')
    convert_stencil(in_directory, out_directory, get_base_name(name)+'-stencil', get_base_name(name), 'png')
    # end = time.time()
    # print('time to generate images', end - start)
    # start = time.time()
    
    outfile = os.path.join(out_directory, '{}.json'.format(res['imagepath']))
    if os.path.exists(outfile):
        return

    if include_entities:
        data = load_entities_data(res['snapshot_id'])

        # end = time.time()
        # print('loading entities data', end - start)
        # start = time.time()
    
        json_entities_data = []
        for i in data:
            json_entity = {
                'model_sizes': i['model_sizes'].tolist(),
                'bbox': i['bbox'].tolist(),
                'pos': i['pos'].tolist(),
                'rot': i['rot'].tolist(),
                'class': i['class'],
                'handle': i['handle'],
                'type': i['type'],
            }
            json_entities_data.append(json_entity)

    # end = time.time()
    # print('dumping entities to json', end - start)
    # start = time.time()

    json_data = {
        'imagepath': res['imagepath'],
        'timestamp': res['timestamp'].strftime("%Y-%m-%d %H:%M:%S"),
        'timeofday': res['timeofday'].strftime("%H:%M:%S"),
        'currentweather': res['currentweather'],
        'width': res['width'],
        'height': res['height'],
        'snapshot_id': res['snapshot_id'],
        'camera_rot': res['camera_rot'].tolist(),
        'camera_pos': res['camera_pos'].tolist(),
        'camera_fov': res['camera_fov'],
        'camera_relative_rotation': res['camera_relative_rotation'].tolist(),
        'camera_relative_position': res['camera_relative_position'].tolist(),
        'view_matrix': res['view_matrix'].tolist(),
        'proj_matrix': res['proj_matrix'].tolist(),
    }
    
    if include_entities:
        json_data['entities']: json_entities_data

    # end = time.time()
    # print('dumping base object to json', end - start)
    # start = time.time()

    with open(outfile, 'w') as f:
        json.dump(json_data, f)
    
    # end = time.time()
    # print('persisting json to file', end - start)
    

### dumping all data

In [47]:
workers = 8

widgets = [progressbar.Percentage(), ' ', progressbar.Counter(), ' ', progressbar.Bar(), ' ',
           progressbar.FileTransferSpeed()]

pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(results)).start()
counter = 0

Parallel(n_jobs=workers, backend='threading')(delayed(dump_snapshot_to_dataset)(in_directory, out_directory, i) for i in results)
print('done')

 16% 10075 |########                                                |  23.8 B/sC:\ProgramData\Anaconda3\lib\site-packages\tifffile\tifffile.py:2642: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)
 16% 10615 |#########                                               |  21.8 B/sC:\ProgramData\Anaconda3\lib\site-packages\tifffile\tifffile.py:2577: UserWarning: unpack: string size must be a multiple of element size
  warnings.warn("unpack: %s" % e)
 99% 62891 |####################################################### |   9.7 B/s

done


### copying to linux into datagrid dir

In [21]:
def copy_file(in_path, out_path):
    if os.path.exists(out_path):
        return
    copyfile(in_path, out_path)

def copy_files(in_dir, out_dir, name):
    if 'pbar' in globals() and 'counter' in globals():
        global counter
        counter += 1
        pbar.update(counter)

    rgb_name = name+'.jpg'
    json_name = name+'.json'
    depth_name = name+'-depth.png'
    stencil_name = name+'-stencil.png'
    
    copyfile(os.path.join(in_dir, rgb_name), os.path.join(out_dir, rgb_name))
    copyfile(os.path.join(in_dir, json_name), os.path.join(out_dir, json_name))
    copyfile(os.path.join(in_dir, depth_name), os.path.join(out_dir, depth_name))
    copyfile(os.path.join(in_dir, stencil_name), os.path.join(out_dir, stencil_name))


workers = 8

windows_in_directory = r'D:\output-datasets\offroad-2'
linux_out_directory = r'Z:\offroad-2'
widgets = [progressbar.Percentage(), ' ', progressbar.Counter(), ' ', progressbar.Bar(), ' ',
           progressbar.FileTransferSpeed()]

pbar = progressbar.ProgressBar(widgets=widgets, max_value=len(results)).start()
counter = 0

Parallel(n_jobs=workers, backend='threading')(
    delayed(copy_files)(windows_in_directory, linux_out_directory, i['imagepath']) for i in results)


100% 9731 |#########################################################|   3.1 B/s

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

### copying original tiffs to linux into datagrid dir, only one run from database

In [42]:
def copy_file(in_path, out_path):
    if os.path.exists(out_path):
        return
    copyfile(in_path, out_path)

def copy_files(in_dir, out_dir, name):
    if 'pbar' in globals() and 'counter' in globals():
        global counter
        counter += 1
        pbar.update(counter)

    rgb_name = name+'.tiff'
    depth_name = name+'-depth.tiff'
    stencil_name = name+'-stencil.tiff'
    
    copyfile(os.path.join(in_dir, rgb_name), os.path.join(out_dir, rgb_name))
    copyfile(os.path.join(in_dir, depth_name), os.path.join(out_dir, depth_name))
    copyfile(os.path.join(in_dir, stencil_name), os.path.join(out_dir, stencil_name))


workers = 8

out_directory = r'Z:\offroad-4-orig'
widgets = [progressbar.Percentage(), ' ', progressbar.Counter(), ' ', progressbar.Bar(), ' ',
           progressbar.FileTransferSpeed()]

pbar = progressbar.ProgressBar(widgets=widgets, max_value=len(results)).start()
counter = 0

Parallel(n_jobs=workers, backend='threading')(
    delayed(copy_files)(in_directory, out_directory, i['imagepath']) for i in results)


  0% 42 |                                                           |   0.7 B/s

KeyboardInterrupt: 

### dumping one snapshot, for testing

In [ ]:
dump_snapshot_to_dataset(in_directory, out_directory, results[0])


In [ ]:
im = np.array(Image.open(os.path.join(out_directory, results[0]['imagepath']+'-depth.png')))
print(np.min(im))
print(np.max(im))


In [ ]:
results[0]['snapshot_id']